In [1]:
import cv2
from scenedetect import VideoManager
from scenedetect import SceneManager
from scenedetect.detectors import ContentDetector
from datetime import datetime
import numpy as np

unrounded_differences = []

def find_subarray_np(main_array, sub_array):
    main_array = np.array(main_array)
    sub_array = np.array(sub_array)
    sub_len = len(sub_array)

    print(main_array)
    print(sub_array)
    
    # Create a view into the main array with the same length as the subarray
    strided = np.lib.stride_tricks.sliding_window_view(main_array, window_shape=sub_len)
    
    # Find all positions where the strided version matches the subarray
    matches = np.all(strided == sub_array, axis=1)
    
    # Get the indices of the matches
    indices = np.where(matches)[0]
    
    if indices.size > 0:
        return indices
    else:
        return -1  # Return -1 if subarray not found

# Function to parse timecodes into timedelta objects
def parse_timecode(time_str):
    # Format: 'HH:MM:SS.sss'
    return datetime.strptime(time_str, "%H:%M:%S.%f")

# Function to compute differences in timestamps
def compute_differences(timestamps):
    # Convert all timestamps to timedelta objects
    times = [parse_timecode(t) for t in timestamps]
    
    # Calculate differences between consecutive timestamps
    differences = []
    for i in range(1, len(times)):
        # Compute difference and convert to total seconds (or any other unit)
        diff = (times[i] - times[i-1]).total_seconds()
        differences.append(round(diff, 2))
    return differences

def find_scenes(video_path, threshold=30.0):
    shot_boundaries = []

    # Create a video manager object for the video.
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    
    # Add the ContentDetector algorithm (with a threshold setting).
    scene_manager.add_detector(ContentDetector(threshold=threshold))
    
    # Start the video manager and perform scene detection.
    video_manager.set_downscale_factor()
    video_manager.start()

    # Detect scenes and return a list of scenes.
    scene_manager.detect_scenes(frame_source=video_manager)
    
    # Obtain the scenes by frame and timecode.
    scene_list = scene_manager.get_scene_list(video_manager.get_base_timecode())
    
    # Each scene is a tuple of (start, end) FrameTimecodes.
    print('List of scene changes:')
    for i, scene in enumerate(scene_list):
        shot_boundaries.append(scene[1].get_timecode())
        #print(f'Scene {i+1}: Start {scene[0].get_timecode()} - End {scene[1].get_timecode()}')

    video_manager.release()
    return shot_boundaries

def query(original_differences):
    query_file_path = '../dataset/queries/joker_query2.mp4'
    query_video_boundaries = find_scenes(query_file_path)
    query_differences = compute_differences(query_video_boundaries)
    return query_differences[:-1]


# Example usage:
video_file_path = '../dataset/originals/joker.mp4'
original_video_boundaries = find_scenes(video_file_path)
original_video_differences = compute_differences(original_video_boundaries)
print(original_video_boundaries)
print(original_video_differences)

query_video_differences = query(original_video_differences)

print(find_subarray_np(original_video_differences, query_video_differences))

c:\Users\Salil\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
VideoManager is deprecated and will be removed.
`base_timecode` argument is deprecated and has no effect.
VideoManager is deprecated and will be removed.


List of scene changes:
['00:00:10.719', '00:00:14.598', '00:00:16.892', '00:00:18.101', '00:00:19.186', '00:00:20.145', '00:00:23.941', '00:00:32.032', '00:00:35.160', '00:00:38.330', '00:00:39.998', '00:00:41.500', '00:00:49.091', '00:00:52.469', '00:01:12.281', '00:01:14.283', '00:01:15.450', '00:01:17.077', '00:01:18.036', '00:01:18.704', '00:01:21.164', '00:01:22.666', '00:01:23.875', '00:01:25.002', '00:01:26.128', '00:01:27.087', '00:01:28.213', '00:01:29.256', '00:01:30.173', '00:01:31.383', '00:01:33.260', '00:01:34.052', '00:01:34.845', '00:01:35.637', '00:01:36.763', '00:01:37.806', '00:01:39.349', '00:01:40.475', '00:01:41.226', '00:01:42.561', '00:01:43.937', '00:01:44.730', '00:01:46.023', '00:01:46.898', '00:01:47.899', '00:01:48.650', '00:01:49.401', '00:01:50.277', '00:01:51.278', '00:01:53.238', '00:02:13.800', '00:02:17.220', '00:02:18.972', '00:02:20.182', '00:02:22.017', '00:02:24.019']
[3.88, 2.29, 1.21, 1.08, 0.96, 3.8, 8.09, 3.13, 3.17, 1.67, 1.5, 7.59, 3.38, 19.

`base_timecode` argument is deprecated and has no effect.


List of scene changes:
[ 3.88  2.29  1.21  1.08  0.96  3.8   8.09  3.13  3.17  1.67  1.5   7.59
  3.38 19.81  2.    1.17  1.63  0.96  0.67  2.46  1.5   1.21  1.13  1.13
  0.96  1.13  1.04  0.92  1.21  1.88  0.79  0.79  0.79  1.13  1.04  1.54
  1.13  0.75  1.33  1.38  0.79  1.29  0.88  1.    0.75  0.75  0.88  1.
  1.96 20.56  3.42  1.75  1.21  1.83  2.  ]
[8.09 3.13 3.17 1.67]
[6]


In [5]:
def find_dominant_color(frame):
    # Split the frame into its RGB channels
    b, g, r = cv2.split(frame)
    
    # Calculate the mean intensity of each channel
    mean_r = np.mean(r)
    mean_g = np.mean(g)
    mean_b = np.mean(b)
    
    # Compare the mean intensities to find the dominant channel
    if mean_r > mean_g and mean_r > mean_b:
        return "R"
    elif mean_g > mean_r and mean_g > mean_b:
        return "G"
    else:
        return "B"

query_file_path = '../dataset/queries/joker_query.mp4'
cap = cv2.VideoCapture(query_file_path)

ret, frame = cap.read()

cap.release()

print(find_dominant_color(frame))

G
